In [4]:
import cv2
import numpy as np

INPUT_WIDTH = 640
INPUT_HEIGHT = 640
SCORE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.45
CONFIDENCE_THRESHOLD = 0.45


In [5]:
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1


In [6]:
BLACK  = (0,0,0)
GREEN   = (0,255,0)
YELLOW = (0,255,255)

In [7]:
def draw_label(im, label, x, y):
    text_size = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    cv2.rectangle(im, (x,y), (x + dim[0], y + dim[1] + baseline), (0,0,0), cv2.FILLED);
    cv2.putText(im, label, (x, y + dim[1]), FONT_FACE, FONT_SCALE, YELLOW, THICKNESS, cv2.LINE_AA)

In [8]:
def pre_process(input_image, net):
      blob = cv2.dnn.blobFromImage(input_image, 1/255,  (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)
      net.setInput(blob)
      outputs = net.forward(net.getUnconnectedOutLayersNames())
      return outputs


In [1]:
def post_process(input_image, outputs):
    class_ids = []
    confidences = []
    boxes = []

    rows = outputs[0].shape[1]
    image_height, image_width = input_image.shape[:2]
    x_factor = image_width / INPUT_WIDTH
    y_factor = image_height / INPUT_HEIGHT

    for r in range (rows):
        row = outputs[0][0][r]
        confidence = row[4]
        if confidence >= CONFIDENCE_THRESHOLD:
            classes_scores = row[5:]
            class_id = np.argmax(classes_scores)
            if (classes_scores[class_id]> SCORE_THRESHOLD):
                confidences.append(class_id)
                cx, cy, w, h = row[0], row[1], row[2], row[3]
                left = int((cx - w/2)* x_factor)
                top = int((cy - h/2)* y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor) 
                box = np.array([left,top,width,height])
                boxes.append(box)
                indices = cv2.dnn.NMSBoxes(boxes,confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
                for i in indices:
                    box = boxes[i]
                    left = box[0]
                    top = box[1]
                    width = box[2]
                    height = box[3]

                    cv2.rectangle(input_image, (left,top), (left+ width, top+height), GREEN, 3*THICKNESS)

                    label = "{}:{:.2f}".format(class_ids[i])
                    draw_label(input_image, label, left, top)
                return input_image


In [ ]:
def main():
    classesFile = "coco.names"
    classes = None
    with open